In [ ]:
# !pip install optuna
# !pip install optuna-integration
# !pip install imutils
# !pip install tensorflow_io

In [ ]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.optimizers import Adam
import tensorflow_io as tfio

import optuna
from optuna.integration import KerasPruningCallback
from optuna.trial import TrialState

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [ ]:
data= tf.keras.utils.image_dataset_from_directory('data', seed=42, image_size=(224, 224), label_mode='binary')
scaled_data = data.map(lambda x, y:(tfio.experimental.color.rgb_to_bgr(x), y))
scaled_data = scaled_data.map(lambda x, y:(x/255, y))

In [ ]:
train_size = int(len(data)*.7)
test_size = int(len(data)*.1)+2
val_size = int(len(data)*.2)
train = scaled_data.take(train_size)
val = scaled_data.skip(train_size).take(val_size)
test = scaled_data.skip(train_size+val_size).take(test_size)

In [ ]:
global train, val

In [ ]:
def create_model(trial):
    # We optimize the number of layers, hidden units and dropout in each layer and
    # the learning rate of RMSProp optimizer.

    vgg_base_model = k.applications.vgg16.VGG16(include_top=False, input_shape=(224,224,3))
    vgg_base_model.trainable=False
    vgg_model = k.Sequential()
    vgg_model.add(vgg_base_model)
    vgg_model.add(k.layers.Flatten())
    vgg_model.add(k.layers.BatchNormalization(scale=False))

    n_layers = trial.suggest_int("n_layers", 1, 3)
    for i in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(i), 64, 512, log=True)
        vgg_model.add(k.layers.Dense(num_hidden, activation="relu"))
        dropout = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        vgg_model.add(k.layers.Dropout(rate=dropout))
        if n_layers!=1:
            vgg_model.add(k.layers.BatchNormalization(scale=False))
        else:
            vgg_model.add(k.layers.BatchNormalization())

    vgg_model.add(k.layers.Dense(1, activation="sigmoid"))

    lr = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)

    vgg_model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=lr),
        metrics=["accuracy"],
    )

    return vgg_model

In [ ]:
def objective(trial):
    # Clear clutter from previous session graphs.
    k.backend.clear_session()


    # Generate our trial model.
    model = create_model(trial)

    # Fit the model on the training data.
    # The KerasPruningCallback checks for pruning condition every epoch.
    model.fit(
        train,
        callbacks=[KerasPruningCallback(trial, "val_accuracy")],
        epochs=10,
        validation_data=val,
        verbose=1,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(val, verbose=0)
    return score[1]

In [ ]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=100)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))